In [250]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [251]:
def mean(df_column):
        return(np.sum(df_column)/len(df_column))
                
def median(df_column):
        df_column = df_column.sort_values(ascending=True).reset_index(drop=True)
        if len(df_column) % 2 == 0:
                return ((df_column[len(df_column)//2]+df_column[len(df_column)//2 - 1])/2)
        else: return df_column[len(df_column)//2]

def mode(df_column):
        freq = {}
        for elem in df_column:
                if elem in freq.keys():
                        freq[elem] += 1
                else : freq[elem] = 1
        return max(freq, key=freq.get)

def tendanceCentrale(df_column): #retourne la moyenne, mode et mediane ainsi que la symétrie
        tendances = []
        mean_ = mean(df_column)
        mode_ = mode(df_column)
        median_ = median(df_column)
        tendances.append(mean_)
        tendances.append(mode_)
        tendances.append(median_)
        if round(mean_,1) == round(mode_,1) and round(mean_,1) == round(median_,1) and round(median_,1) == round(mode_,1):
                tendances.append("Symétrique")
        elif round(mean_,1) > round(median_,1) and round(median_,1) > round(mode_,1):
                tendances.append("Positivement")
        elif round(mean_,1) < round(median_,1) and round(median_,1) < round(mode_,1):
                tendances.append("Négativement")
        else: tendances.append("Inindentifiée")
        return pd.Series(np.array(tendances), index=['mean', 'mode', 'median', 'symetrie'])

In [252]:
def ecartType(df_column):
        return(np.sqrt((np.sum(np.power(df_column-mean(df_column),2)))/len(df_column)))

def variance(df_column):
        return(np.power(ecartType(df_column),2))

def getQuartiles(df_column):
        df_column = df_column.sort_values(ascending=True).reset_index(drop=True)
        return (df_column[len(df_column)//4], df_column[(len(df_column)//4)*3])
        
def ecartInterquartile(df_column):
        Q1, Q3 = getQuartiles(df_column)
        return Q3-Q1
        
def dispersion(df_column): #get outlier data and make it into a pandas series
        mesures = []
        standard_deviation = ecartType(df_column)
        variance_ = variance(df_column)
        inter_q = ecartInterquartile(df_column)
        quart = getQuartiles(df_column)
        mesures.append(standard_deviation)
        mesures.append(variance_)
        mesures.append(inter_q)
        mesures.append(min(df_column))
        mesures.append(quart)
        mesures.append(max(df_column))
        outliers = []
        for each in df_column:
                if each > quart[1]+1.5*inter_q or each < quart[0]-1.5*inter_q:
                        outliers.append(each)
        return (pd.Series(np.array(mesures), index=['ecart_type', 'variance', 'IQR', 'min','quartiles', 'max']), set(outliers))

In [253]:

def z_score(dataframe):
        dataframe = pd.DataFrame(dataframe)
        z_scores = []
        means = []
        std_devs = []
        for j in dataframe.columns:
                means.append(mean(dataframe[[j]]).values[0])
                std_devs.append(ecartType(dataframe[[j]]).values[0])
        for i in range(0, dataframe.shape[0]):
                row = []
                c = 0
                for j in dataframe.columns:
                        row.append(((dataframe[[j]].iloc[i]-means[c])/std_devs[c]).values[0])
                        c+=1
                z_scores.append(row)
        normalized = pd.DataFrame(z_score)
        return normalized
        

In [254]:
def correlation(df_column1, df_column2):
        try:
                N = np.int64(len(df_column1))
                xy_sum = np.int64(np.sum(df_column1*df_column2))
                top_sum = np.int64(N*xy_sum)
                sum_x = np.sum(df_column1)
                sum_y = np.sum(df_column2)
                bottom_x = len(df_column1)*np.sum(list(map(lambda x: x*x, df_column1)))
                bottom_y = len(df_column2)*np.sum(list(map(lambda x: x*x, df_column2)))
                return ((top_sum - (sum_x*sum_y))/np.sqrt((bottom_x - np.power(sum_x,2))*(bottom_y - np.power(sum_y,2))))
        except: print("impossible to calculate")

In [255]:
df = pd.read_csv("dataset1.csv")
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [256]:
data_num = df.select_dtypes(exclude=['object'])
data_obj = df.select_dtypes(exclude=['float64', 'int64'])
data_num.head()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1102,1,2,1,1,2.0,94,3,2,...,1,80,0,8,0,1,6,4,0,5
1,49,279,8,1,1,2,3.0,61,2,2,...,4,80,1,10,3,3,10,7,1,7
2,37,1373,2,2,1,4,4.0,92,2,1,...,2,80,0,7,3,3,0,0,0,0
3,33,1392,3,4,1,5,4.0,56,3,1,...,3,80,0,8,3,3,8,7,3,0
4,27,591,2,1,1,7,1.0,40,3,1,...,4,80,1,6,3,3,2,2,2,2


In [257]:
data_obj.head()

,Attrition,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,Over18,OverTime
0,Yes,Travel_Rarely,Sales,Life Sciences,Female,Sales Executive,Single,Y,Yes
1,No,Travel_Frequently,Research & Development,Life Sciences,Male,Research Scientist,Married,Y,No
2,Yes,Travel_Rarely,Research & Development,Other,Male,Laboratory Technician,Single,Y,Yes
3,No,Travel_Frequently,Research & Development,Life Sciences,Female,Research Scientist,Married,Y,Yes
4,No,Travel_Rarely,Research & Development,Medical,Male,Laboratory Technician,Married,Y,No


In [258]:
mode(data_obj.Attrition)

'No'

### A. Traitement des valeurs manquantes et aberrantes

In [259]:
def replace_missing(df_column, method = 'mode'):
        print(df_column.dtype)
        if df_column.dtype == 'object':
                if method != 'mode': print('this method cannot be used on non-numerical attributes')
                else: 
                        for row in df_column.items():
                                if pd.isnull(row[1]) : 
                                        if method == 'mode':
                                                df_column[row[0]] = mode(df_column)
        else:
                for row in df_column.items():
                        #print(df_column[0])
                        if pd.isnull(row[1]) : 
                                if method == 'mode':
                                        df_column[row[0]] = mode(df_column)
                                elif method == 'median':
                                        df_column[row[0]] = median(df_column)
                                elif method == 'mean':
                                        df_column[row[0]] = mean(df_column)
                                else: print("an error has occured")
        return df_column

def treat_outliers(dataframe, df_column, method = "delete"):
        outliers = dispersion(df_column)[1]
        df = dataframe.copy()
        #iterate through all rows
        for i in range(len(df_column)):
                if df_column.iloc[i] in outliers : 
                        if method == "delete":
                                df=df.drop([i])
                        elif method == "null":
                                df_column.iloc[i] = np.nan
                                df[df_column] = df_column
                        elif method == "replace_mode":
                                df_column.iloc[i] = mode(df_column)
                                df[df_column] = df_column
                        elif method == "replace_mean":
                                if df_column.dtype == "int64":
                                        df_column.iloc[i] = round(mean(df_column))
                                else: df_column.iloc[i] = mean(df_column)
                                df[df_column] = df_column
                        elif method == "replace_median":
                                df_column.iloc[i] = median(df_column)
                                df[df_column] = df_column
                        else: print("la methode choisie est invalide")

        return df


In [260]:
#replace_missing(df.EnvironmentSatisfaction).isnull().sum()
#treat_outliers(df, df.YearsInCurrentRole,method = "null")

### B. Réduction des données via la discretisation des données continues

In [261]:
# divide the dataset into n quantiles of the same amount of values/rows
# this means each group will have the same amount of values but different interval sizes
# we can reduce a group into a single row by either doing the mean, the median or the mode
def discretisation_effectifs(df_column, Q, method = "mean"):
        step = (len(df_column)//Q)+1
        column = df_column.sort_values(ascending=True).reset_index(drop=True)
        quantiles = []
        for i in range(0,len(df_column),step):
                quantiles.append(column.iloc[i:i+step])
        for i in range(len(quantiles)): 
                if method == "mean":
                        if df_column.dtype == "int64":
                                quantiles[i] = round(mean(quantiles[i]))
                        else:
                                quantiles[i] = mean(quantiles[i])
                elif method == "mode":
                        quantiles[i] = mode(quantiles[i])
                elif method == "median":
                        quantiles[i] = median(quantiles[i])
                else: print("la methode choisie est invalide")

        # we return the reduced column and then we just make another function to loop over the df?
        # but in this case we need to ask the chosen method for each column so idk about that lol 
        # they must also have the same number of quantiles too so
        return quantiles

In [262]:
discretisation_effectifs(df.Age, 20, method = "mean")

[21,
 25,
 27,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 38,
 39,
 40,
 42,
 44,
 46,
 49,
 53,
 57]

In [263]:
# divide the dataset into n intervals of values
# this means each group will have the same interval sizes but different amount of values
# we can reduce a group into a single row by either doing the mean, the median or the mode
# if k value is not given, use the formula of huntsberger
import math
def discretisation_amplitude(df_column, K = 0, method = "mean"):
        if K==0:
                K = 1+3*math.log10(len(df_column))
        ## column = df_column.sort_values(ascending=True).reset_index(drop=True)
        interval_size = (max(df_column)- min(df_column))/K
        if df_column.dtype == "int64": interval_size = round(interval_size)
        intervals = []
        i = min(df_column)
        while i < max(df_column):
                intervals.append((i, i+interval_size))
                i+=interval_size
        intervals_values = {}
        for each in df_column:
                for interval in intervals:
                        if each > interval[0] and each < interval[1]:
                                if interval in intervals_values.keys():
                                        intervals_values[interval].append(each)
                                else: 
                                        intervals_values[interval] = [each]
        discretised = []
        for interval in intervals_values.keys():
                if method == "mean":
                        if df_column.dtype == "int64":
                                discretised.append(round(mean(intervals_values[interval])))
                        else: discretised.append(mean(intervals_values[interval]))
                elif method == "median":
                        discretised.append(median(intervals_values[interval]))
                elif method == "mode":
                        discretised.append(mode(intervals_values[interval]))
                else: print("methode choisie invalide")
        discretised.sort()
        return discretised

In [264]:
discretisation_amplitude(df.Age)

[20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 59]

### C. Réduction des données (redondances) horizontales et verticales

In [265]:
def detect_redundant(df):
        redundances = df.duplicated()
        redundant_columns = []
        for each in df.columns:
                if len(df[each].unique()) == 1 or len(df[each].unique()) == df.shape[0]:
                        redundant_columns.append(each)
        return df[redundances], redundant_columns


In [266]:
df.shape

(1470, 35)

In [267]:
detect_redundant(df)

(Empty DataFrame
 Columns: [Age, Attrition, BusinessTravel, DailyRate, Department, DistanceFromHome, Education, EducationField, EmployeeCount, EmployeeNumber, EnvironmentSatisfaction, Gender, HourlyRate, JobInvolvement, JobLevel, JobRole, JobSatisfaction, MaritalStatus, MonthlyIncome, MonthlyRate, NumCompaniesWorked, Over18, OverTime, PercentSalaryHike, PerformanceRating, RelationshipSatisfaction, StandardHours, StockOptionLevel, TotalWorkingYears, TrainingTimesLastYear, WorkLifeBalance, YearsAtCompany, YearsInCurrentRole, YearsSinceLastPromotion, YearsWithCurrManager]
 Index: []
 
 [0 rows x 35 columns],
 ['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'])

In [268]:
def del_row(df, index):
        return df.drop([index], axis = 0)

def del_column(df, index):
        if type(index) is str:
                if index in df.columns:
                        df = df.drop([index], axis = 1)
        elif type(int(index)) is int:
                df = df.drop(df.columns[index], axis = 1)
        return df


In [269]:
#del_column(df, "Age")

### D. Normalisation des données

In [273]:
# follow formula
def min_max_normalisation(df, min_, max_):
        if max_ < min_ : 
                print("min superior to max, please change values")
                return df
        else:
                normalized_df = []
                #df = df.select_dtypes(exclude=['object'])
                min_old = []
                max_old = []
                for column in df.columns:
                        min_old.append(min(df[column]))
                        max_old.append(max(df[column]))
                for i in range(df.shape[0]):
                        row = []
                        c = 0
                        for column in df.columns:
                                if df[column].dtype == 'object':
                                        row.append(df[column].iloc[i])
                                else: row.append((((df[column].iloc[i]-min_old[c])/(max_old[c]- min_old[c]))*(max_- min_))+min_)  
                                c+=1
                        normalized_df.append(row)
                normalized = pd.DataFrame(normalized_df, columns = df.columns)
        return normalized
                                
                        

In [274]:
min_max_normalisation(df, 2,5).isnull().sum()

C:\Users\YsPC\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in longlong_scalars


Age                            0
Attrition                      0
BusinessTravel                 0
DailyRate                      0
Department                     0
DistanceFromHome               0
Education                      0
EducationField                 0
EmployeeCount               1470
EmployeeNumber                 0
EnvironmentSatisfaction       12
Gender                         0
HourlyRate                     0
JobInvolvement                 0
JobLevel                       0
JobRole                        0
JobSatisfaction                0
MaritalStatus                  0
MonthlyIncome                  8
MonthlyRate                    0
NumCompaniesWorked             0
Over18                         0
OverTime                       0
PercentSalaryHike              0
PerformanceRating              0
RelationshipSatisfaction       0
StandardHours               1470
StockOptionLevel               0
TotalWorkingYears              0
TrainingTimesLastYear          0
WorkLifeBa

In [272]:
#replace with the z score values, already done really but double check if it's the same formula
def z_score_normalisation(df):
        pass      
def z_score_normalisation(dataframe):
        dataframe = pd.DataFrame(dataframe)
        #dataframe = dataframe.select_dtypes(exclude=['object'])
        z_scores = []
        means = []
        std_devs = []
        for j in dataframe.columns:
                means.append(mean(dataframe[[j]]).values[0])
                std_devs.append(ecartType(dataframe[[j]]).values[0])
        for i in range(0, dataframe.shape[0]):
                row = []
                c = 0
                for j in dataframe.columns:
                        if df[j].dtype == 'object':
                                row.append(df[j].iloc[i])
                        else: row.append(((dataframe[[j]].iloc[i]-means[c])/std_devs[c]).values[0])
                        c+=1
                z_scores.append(row)
        normalized = pd.DataFrame(z_score, columns = dataframe.columns)
        return normalized
        